In [ ]:
!pip install -U accelerate
!pip install -U transformers
!pip install --upgrade huggingface_hub
!pip install evaluate

In [ ]:
from huggingface_hub import login
login()

In [ ]:
from google.colab import files

uploaded = files.upload()
fnames = []
for fn in uploaded.keys():
  fnames.append(fn)
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [ ]:
print("nosql_injection_dataset.json")

In [ ]:
import gc
import torch
torch.cuda.empty_cache()
gc.collect()

In [ ]:
import torch
import accelerate
from transformers import AutoModelForMaskedLM, AutoTokenizer, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
from torch.utils.data import Dataset, DataLoader
import json

# Define the dataset class
import torch
from torch.utils.data import Dataset, TensorDataset

import torch
from torch.utils.data import Dataset

class NoSQLInjectionDataset(Dataset):
    def __init__(self, dataset, tokenizer, split_ratio=0.9):
        self.dataset = dataset
        self.tokenizer = tokenizer
        self.split_ratio = split_ratio
        self.train_dataset = self.dataset[:int(len(self.dataset) * self.split_ratio)]
        self.eval_dataset = self.dataset[int(len(self.dataset) * self.split_ratio):]

    def __len__(self):
        return len(self.train_dataset)

    def __getitem__(self, idx):
        sample = self.train_dataset[idx]
        template = sample["template"]
        payload = sample["payload"]

        # Tokenize template and payload
        tokenized_inputs = self.tokenizer.encode_plus(
            template,
            payload,
            truncation=True,
            padding="max_length",
            max_length=512,
            return_tensors="pt"
        )

        input_ids = tokenized_inputs["input_ids"].squeeze()
        attention_mask = tokenized_inputs["attention_mask"].squeeze()

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": input_ids
        }

    def get_eval_dataset(self):
        eval_dataset = []
        for sample in self.eval_dataset:
            template = sample["template"]
            payload = sample["payload"]

            tokenized_inputs = self.tokenizer.encode_plus(
                template,
                payload,
                truncation=True,
                padding="max_length",
                max_length=512,
                return_tensors="pt"
            )

            input_ids = tokenized_inputs["input_ids"].squeeze()
            attention_mask = tokenized_inputs["attention_mask"].squeeze()

            eval_dataset.append({
                "input_ids": input_ids,
                "attention_mask": attention_mask,
                "labels": input_ids
            })

        return eval_dataset
with open("nosql_injection_dataset.json", "r") as f:
    dataset = json.load(f)
tokenizer = AutoTokenizer.from_pretrained("distilroberta-base")
tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)
# Load the tokenizer and model
model = AutoModelForMaskedLM.from_pretrained("distilroberta-base")
# Convert the dataset into a PyTorch Dataset
train_dataset = NoSQLInjectionDataset(dataset, tokenizer)
eval_dataset = train_dataset.get_eval_dataset()
# Set up the training arguments and trainer
training_args = TrainingArguments(
    output_dir="fine-tuned-distilroberta-nosql-injection",
    evaluation_strategy="epoch",
    num_train_epochs=75, 
    per_device_train_batch_size=8,
    save_steps=1000, 
    logging_steps=100, # used to adjust the frequency of logging loss values
    logging_dir="./logs",
    overwrite_output_dir=True,
    learning_rate=2e-5,  # Adjust as needed
    warmup_steps=100,  # Adjust as needed
    weight_decay=0.01,  # Adjust as needed
    push_to_hub=True
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
)

# Fine-tune the model
trainer.train()

# Save the fine-tuned model

# Push the fine-tuned model to the Hugging Face Model Hub
trainer.push_to_hub("fine-tuned-distilroberta-nosql-injection")
tokenizer.push_to_hub("fine-tuned-distilroberta-nosql-injection")
print("Fine-tuned model saved and pushed to the Hugging Face Model Hub.")

In [ ]:
trainer.push_to_hub("fine-tuned-distilroberta-nosql-injection")
tokenizer.push_to_hub("fine-tuned-distilroberta-nosql-injection")
print("Fine-tuned model saved and pushed to the Hugging Face Model Hub.")

In [ ]:
import math

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")
trainer.push_to_hub()
tokenizer.push_to_hub()

In [ ]:
from transformers import pipeline

mask_filler = pipeline("fill-mask", "ankush-003/fine-tuned-distilroberta-nosql-injection")

mask_filler("{username: <mask>, password: anksh}", top_k=3)

In [ ]:
!tensorboard --logdir=logs

## Trying roberta

In [1]:
from transformers import AutoTokenizer, RobertaForMaskedLM, DataCollatorForLanguageModeling
import torch
import accelerate
from transformers import Trainer, TrainingArguments
from torch.utils.data import Dataset, DataLoader
import json

class NoSQLInjectionDataset(Dataset):
    def __init__(self, dataset, tokenizer, split_ratio=0.9):
        self.dataset = dataset
        self.tokenizer = tokenizer
        self.split_ratio = split_ratio
        self.train_dataset = self.dataset[:int(len(self.dataset) * self.split_ratio)]
        self.eval_dataset = self.dataset[int(len(self.dataset) * self.split_ratio):]

    def __len__(self):
        return len(self.train_dataset)

    def __getitem__(self, idx):
        sample = self.train_dataset[idx]
        template = sample["template"]
        payload = sample["payload"]

        # Tokenize template and payload
        tokenized_inputs = self.tokenizer.encode_plus(
            template,
            payload,
            truncation=True,
            padding="max_length",
            max_length=512,
            return_tensors="pt"
        )

        input_ids = tokenized_inputs["input_ids"].squeeze()
        attention_mask = tokenized_inputs["attention_mask"].squeeze()

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": input_ids
        }

    def get_eval_dataset(self):
        eval_dataset = []
        for sample in self.eval_dataset:
            template = sample["template"]
            payload = sample["payload"]

            tokenized_inputs = self.tokenizer.encode_plus(
                template,
                payload,
                truncation=True,
                padding="max_length",
                max_length=512,
                return_tensors="pt"
            )

            input_ids = tokenized_inputs["input_ids"].squeeze()
            attention_mask = tokenized_inputs["attention_mask"].squeeze()

            eval_dataset.append({
                "input_ids": input_ids,
                "attention_mask": attention_mask,
                "labels": input_ids
            })

        return eval_dataset
with open("nosql_injection_dataset.json", "r") as f:
    dataset = json.load(f)

tokenizer = AutoTokenizer.from_pretrained("roberta-base")
tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

train_dataset = NoSQLInjectionDataset(dataset, tokenizer)
eval_dataset = train_dataset.get_eval_dataset()

model = RobertaForMaskedLM.from_pretrained("roberta-base")

# Set up the training arguments and trainer
training_args = TrainingArguments(
    output_dir="fine-tuned-roberta-nosql-injection",
    evaluation_strategy="epoch",
    num_train_epochs=75, 
    per_device_train_batch_size=4,
    save_steps=1000, 
    logging_steps=100, # used to adjust the frequency of logging loss values
    logging_dir="./logs",
    overwrite_output_dir=True,
    learning_rate=2e-5,  # Adjust as needed
    warmup_steps=100,  # Adjust as needed
    weight_decay=0.01,  # Adjust as needed
    push_to_hub=True
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
)

# Fine-tune the model
trainer.train()

# Save the fine-tuned model

# Push the fine-tuned model to the Hugging Face Model Hub
trainer.push_to_hub("fine-tuned-roberta-nosql-injection")
tokenizer.push_to_hub("fine-tuned-roberta-nosql-injection")
print("Fine-tuned model saved and pushed to the Hugging Face Model Hub.")

2023-07-18 19:31:39.146214: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-18 19:31:39.650764: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/gariman/Downloads/angus/fine-tuned-roberta-nosql-injection is already a clone of https://huggingface.co/ankush-003/fine-tuned-roberta-nosql-injection. Make sure you pull the latest changes with `repo.git_pull()`.
/home/gariman/anaconda3/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/11850 [00:00<?, ?it/s]

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'loss': 1.2572, 'learning_rate': 2e-05, 'epoch': 0.63}


  0%|          | 0/9 [00:00<?, ?it/s]

...

{'eval_loss': 6.927690066049763e-08, 'eval_runtime': 2.0219, 'eval_samples_per_second': 34.621, 'eval_steps_per_second': 4.451, 'epoch': 61.0}
{'loss': 0.0005, 'learning_rate': 3.6595744680851063e-06, 'epoch': 61.39}


  0%|          | 0/9 [00:00<?, ?it/s]

{'eval_loss': 0.030300242826342583, 'eval_runtime': 2.0196, 'eval_samples_per_second': 34.661, 'eval_steps_per_second': 4.456, 'epoch': 62.0}
{'loss': 0.0158, 'learning_rate': 3.48936170212766e-06, 'epoch': 62.03}
{'loss': 0.005, 'learning_rate': 3.3191489361702127e-06, 'epoch': 62.66}


  0%|          | 0/9 [00:00<?, ?it/s]

{'eval_loss': 5.990250429022126e-08, 'eval_runtime': 2.0842, 'eval_samples_per_second': 33.585, 'eval_steps_per_second': 4.318, 'epoch': 63.0}
{'loss': 0.0001, 'learning_rate': 3.1489361702127664e-06, 'epoch': 63.29}
